# 4. Structured Streaming with the Retail Data-Set #
Use the by-day retail data-set to simulate a daily feed of data to be read by a Spark Structured Streaming job.   

Each file that is read in by the `spark.ReadStream` method uses the option *maxFilesPerTrigger=1* to trigger an update to the streaming dataframe contents.  
  
A transformation called `purchaseByCustomerPerHour` is mapped to the streaming dataframe with a query transformation to show the top 5 customers and their spend so far. 

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook-4").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        config("spark.eventLog.enabled", "true").\
        config("spark.eventLog.dir", "file:///opt/workspace/events").\
        getOrCreate()      

In [2]:
# Spark Streaming functions for creating window and defining a column (col) to calculate the window over
#from pyspark.sql.functions import window, column, desc, col
from pyspark.sql.functions import window,  col

First, get a sample of the data to be processed in a continuous stream.  
+ We have to assume that future data will have the same schema.  

In [3]:
data_sample = spark.read.option("inferSchema", True).option("header", True).csv("/opt/workspace/datain/retail-data/by-day/2010-12-01.csv")

In [4]:
data_sample.take(5)

[Row(InvoiceNo='536365', StockCode='85123A', Description='WHITE HANGING HEART T-LIGHT HOLDER', Quantity=6, InvoiceDate=datetime.datetime(2010, 12, 1, 8, 26), UnitPrice=2.55, CustomerID=17850.0, Country='United Kingdom'),
 Row(InvoiceNo='536365', StockCode='71053', Description='WHITE METAL LANTERN', Quantity=6, InvoiceDate=datetime.datetime(2010, 12, 1, 8, 26), UnitPrice=3.39, CustomerID=17850.0, Country='United Kingdom'),
 Row(InvoiceNo='536365', StockCode='84406B', Description='CREAM CUPID HEARTS COAT HANGER', Quantity=8, InvoiceDate=datetime.datetime(2010, 12, 1, 8, 26), UnitPrice=2.75, CustomerID=17850.0, Country='United Kingdom'),
 Row(InvoiceNo='536365', StockCode='84029G', Description='KNITTED UNION FLAG HOT WATER BOTTLE', Quantity=6, InvoiceDate=datetime.datetime(2010, 12, 1, 8, 26), UnitPrice=3.39, CustomerID=17850.0, Country='United Kingdom'),
 Row(InvoiceNo='536365', StockCode='84029E', Description='RED WOOLLY HOTTIE WHITE HEART.', Quantity=6, InvoiceDate=datetime.datetime(20

#### Create a Streaming Dataframe ####
The Streaming Dataframe is created with a Schema based on the data sample we took earlier.

In [5]:
# Get a Schema from the data_sample
staticSchema = data_sample.schema

Create a streaming dataframe using `maxFilesPerTrigger` to trigger an update to the dataframe on file-by-file basis.  The `header` option caters for (assumes) headers on each file that is loaded.

In [6]:
# create a Streaming DataFrame based on a Static Schema from the data_sample
streamingDataFrame = spark.readStream.schema(staticSchema).option("maxFilesPerTrigger",1).format("csv").option("header", "true")\
       .load("/opt/workspace/datain/retail-data/by-day/*.csv")

In [7]:
# check the stream status
streamingDataFrame.isStreaming

True

A streaming transformation `purchaseByCustomerPerHour` is created below; This applies a dataframe transformation which groups the data by *CustomerID* and *InvoiceDate* and sums the "total_cost" of *UnitPrice* x *Quantity*.  

The `window` function is used to define the size of window over which this summary is provided - in this case a 1-day window of time.  This means we get to see which customers have bought the most *in a single day*

In [8]:
# Create a transformation that sum's customer purchase per hour
purchaseByCustomerPerHour = streamingDataFrame.selectExpr("CustomerID", "(UnitPrice * Quantity) as total_cost", "InvoiceDate") \
                                              .groupBy(col("CustomerID"), window(col("InvoiceDate"), "1 day")) \
                                              .sum("total_cost")

Create an in-memory structure called `customer_purchases` that  is written to with the output from the streaming transformation. This can be queried using Spark SQL to see the latest result-set.

In [9]:
# Generate updates to an in-memory table after each trigger
purchaseByCustomerPerHour.writeStream.format("memory").queryName("customer_purchases").outputMode("complete").start()

Query `customer_purchases` to see the latest view of which customers have spent the most in a single day.  The results change as new retail data day-files are processed.

In [20]:
spark.sql("""
SELECT * FROM customer_purchases
ORDER BY `sum(total_cost)` DESC
""").show(5)

+----------+--------------------+------------------+
|CustomerID|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   18102.0|[2010-12-07 00:00...|          25920.37|
|      null|[2010-12-10 00:00...|25399.560000000012|
|      null|[2010-12-17 00:00...|25371.769999999768|
|      null|[2010-12-06 00:00...|23395.099999999904|
|      null|[2010-12-03 00:00...| 23021.99999999999|
+----------+--------------------+------------------+
only showing top 5 rows



In [21]:
spark.stop()